In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset
import numpy as np
from src.evaluate_crows import evaluate_crows
import os
import pickle

In [16]:
#models_names = ['distilbert/distilbert-base-uncased','albert/albert-base-v2']
models_names = ['albert/albert-base-v2','FacebookAI/xlm-roberta-base']

# Crows evaluation

In [ ]:

for model_name in models_names:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    print('Computing CROWS for: ',model_name,)
    results = evaluate_crows(model,tokenizer)
    result_path = 'results/'+model_name.replace('/','-')
    os.makedirs(result_path, exist_ok=True)
    with open(os.path.join(result_path,'crows.pkl'),'wb') as f:
        pickle.dump(results, f)
    print('-'*25)



Some weights of the model checkpoint at albert/albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Computing CROWS for:  albert/albert-base-v2


100%|██████████| 1508/1508 [12:42<00:00,  1.98it/s]


-------------------------


Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Computing CROWS for:  FacebookAI/xlm-roberta-base


100%|██████████| 1508/1508 [27:41<00:00,  1.10s/it]

-------------------------


In [20]:
with open('results/albert-albert-base-v2/crows.pkl','rb') as f:
    result_alb = pickle.load(f)

In [21]:
result_alb

{'age': np.float64(0.5172413793103449),
 'disability': np.float64(0.7166666666666667),
 'gender': np.float64(0.4198473282442748),
 'nationality': np.float64(0.4276729559748428),
 'physical-appearance': np.float64(0.5714285714285714),
 'race-color': np.float64(0.5251937984496124),
 'religion': np.float64(0.5428571428571428),
 'sexual-orientation': np.float64(0.7738095238095238),
 'socioeconomic': np.float64(0.6918604651162791),
 'overall': np.float64(0.5397877984084881)}

In [22]:
results

{'age': np.float64(0.4942528735632184),
 'disability': np.float64(0.7666666666666667),
 'gender': np.float64(0.5343511450381679),
 'nationality': np.float64(0.4716981132075472),
 'physical-appearance': np.float64(0.5238095238095238),
 'race-color': np.float64(0.5232558139534884),
 'religion': np.float64(0.6571428571428571),
 'sexual-orientation': np.float64(0.5714285714285714),
 'socioeconomic': np.float64(0.6104651162790697),
 'overall': np.float64(0.5497347480106101)}